In [3]:
import pandas as pd
import os
import random
from datetime import datetime as dt

In [5]:
data_dir = '/opt/ml/input/data' # 경로는 상황에 맞춰서 수정해주세요!
csv_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
df = pd.read_csv(csv_file_path)
df['Timestamp'] = df['Timestamp'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [6]:
print("-------------------DKT_information--------------------")
print(f"row 수 : {df.shape[0]}")
print(f"유저 수 : {df['userID'].nunique()}")
print(f"문제 수 : {df['assessmentItemID'].nunique()}")
print(f"문제지 수 : {df['testId'].nunique()}")
print(f"태그 수 : {df['KnowledgeTag'].nunique()}")
print(f"유저-문제 상호작용 수 :{df[['userID','assessmentItemID','testId']].drop_duplicates().shape}")
print(f"희소율 : { (1 - round(df[['userID','assessmentItemID','testId']].drop_duplicates().shape[0] / (df['userID'].nunique()*df['assessmentItemID'].nunique()),4))*100}%")
print("------------------------------------------------------")

-------------------DKT_information--------------------
row 수 : 2266586
유저 수 : 6698
문제 수 : 9454
문제지 수 : 1537
태그 수 : 912
유저-문제 상호작용 수 :(2220633, 3)
희소율 : 96.49%
------------------------------------------------------


In [66]:
df.head(2)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,month,category_2
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,3,6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3,6


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2266586 entries, 0 to 2266585
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   userID            int64 
 1   assessmentItemID  object
 2   testId            object
 3   answerCode        int64 
 4   Timestamp         object
 5   KnowledgeTag      int64 
dtypes: int64(3), object(3)
memory usage: 103.8+ MB


In [70]:
df_070 = df[df['assessmentItemID'].str[1:4]=='070']

In [78]:
df_070['47'] = df_070['assessmentItemID'].str[4:7]

/tmp/ipykernel_11695/200517804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_070['47'] = df_070['assessmentItemID'].str[4:7]


In [98]:
t = df_070.groupby('47')['answerCode']
pd.DataFrame({'mean':t.mean(),'cnt':t.count()}).reset_index().sort_values(by='mean')

,47,mean,cnt
74,075,0.352535,1302
38,039,0.359690,1290
58,059,0.370031,1308
140,141,0.374434,1768
62,063,0.384793,1302
...,...,...,...
136,137,0.696483,1308
27,028,0.708889,1350
37,038,0.719419,1308
95,096,0.728379,1561


In [48]:
df['Timestamp'].dt.month.value_counts()

9     339265
10    327226
8     316115
7     270597
6     227481
5     183816
4     176953
3     174482
2      79478
11     71695
12     62091
1      37387
Name: Timestamp, dtype: int64

In [50]:
df['month'] = df['Timestamp'].dt.month

In [55]:
df.groupby('month')['answerCode'].mean().sort_values()
# 월별로 다른 이유는 수가 너무 적기 때문?

month
12    0.496304
11    0.505544
5     0.643317
4     0.649545
6     0.651967
8     0.657599
7     0.665650
9     0.668872
10    0.670372
3     0.682643
2     0.687222
1     0.712199
Name: answerCode, dtype: float64

In [58]:
df['category_2'] = df['assessmentItemID'].str[2]
temp = df.groupby('category_2')['answerCode']
pd.DataFrame({'mean':temp.mean(),'cnt':temp.count()}).reset_index().sort_values('mean')

,category_2,mean,cnt
8,9,0.449948,119385
7,8,0.502598,246336
6,7,0.521876,279164
4,5,0.658208,275773
3,4,0.684056,267323
2,3,0.702238,273762
5,6,0.709232,264434
1,2,0.737593,268327
0,1,0.800876,272082


In [26]:
df

,userID,assessmentItemID,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,1,2020-03-24 00:17:11,7224
1,0,A060001002,1,2020-03-24 00:17:14,7225
2,0,A060001003,1,2020-03-24 00:17:22,7225
3,0,A060001004,1,2020-03-24 00:17:29,7225
4,0,A060001005,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...
2266581,7441,A030071005,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,1,2020-08-21 01:07:36,8836


## 대,중,소 분류로 나누고 시작

In [23]:
df.drop('testId',axis=1,inplace=True) # TestID 날림.

In [29]:
df[:2]

,userID,assessmentItemID,answerCode,Timestamp,KnowledgeTag,1,2,3
0,0,A060001001,1,2020-03-24 00:17:11,7224,060,001,001
1,0,A060001002,1,2020-03-24 00:17:14,7225,060,001,002


In [63]:
a = df[['assessmentItemID']].drop_duplicates()

In [65]:
a['1'] = a['assessmentItemID'].str[1:4]
a['2'] = a['assessmentItemID'].str[4:7]
a['3'] = a['assessmentItemID'].str[7:10]

## 대분류(1-4)

In [80]:
a[['1','2']].drop_duplicates().groupby('1')['2'].count().sort_values() # 대분류당 문제지 수

1
090     74
080    138
070    164
060    186
010    192
050    193
020    194
030    198
040    198
Name: 2, dtype: int64

In [81]:
a[['1','2','3']].drop_duplicates().groupby('1')['3'].count().sort_values() # 대분류당 문제 수 

1
090     515
010     975
020    1022
040    1031
080    1050
030    1060
060    1213
050    1266
070    1322
Name: 3, dtype: int64

In [84]:
df['assessmentItemID'].str[1:4].value_counts() # 대분류당 유저가 푼 문제 수

070    279164
050    275773
030    273762
010    272082
020    268327
040    267323
060    264434
080    246336
090    119385
Name: assessmentItemID, dtype: int64

### 090만 문제지 수와 문제 수가 적은 것 빼곤 특이사항 없음.

In [85]:
df.groupby('1')['answerCode'].mean().sort_values(ascending=False)

1
010    0.800876
020    0.737593
060    0.709232
030    0.702238
040    0.684056
050    0.658208
070    0.521876
080    0.502598
090    0.449948
Name: answerCode, dtype: float64

### 060을 제외하고는 점점 정답률이 낮아지므로 난이도로 생각할 수 있다.

## 중분류(4~7)

In [38]:
pd.set_option('display.max_row', 200)

In [86]:
df[:2]

,userID,assessmentItemID,answerCode,Timestamp,KnowledgeTag,1,2,3
0,0,A060001001,1,2020-03-24 00:17:11,7224,060,001,001
1,0,A060001002,1,2020-03-24 00:17:14,7225,060,001,002


In [106]:
a.groupby('2')['3'].count() # 중분류별 문제 개수

2
001    56
002    57
003    58
004    58
005    57
006    57
007    59
008    58
009    60
010    60
011    58
012    57
013    56
014    54
015    56
016    59
017    64
018    65
019    56
020    55
021    54
022    56
023    56
024    57
025    59
026    51
027    54
028    51
029    55
030    52
031    55
032    56
033    61
034    59
035    60
036    60
037    56
038    54
039    60
040    54
041    52
042    52
043    61
044    64
045    56
046    56
047    58
048    57
049    56
050    56
051    57
052    59
053    52
054    53
055    57
056    59
057    64
058    62
059    55
060    55
061    57
062    58
063    54
064    51
065    55
066    56
067    60
068    62
069    54
070    60
071    57
072    55
073    56
074    56
075    47
076    47
077    49
078    49
079    52
080    49
081    47
082    46
083    49
084    48
085    47
086    53
087    52
088    52
089    49
090    53
091    51
092    51
093    45
094    50
095    45
096    49
097    48
098    49
099    51
100    5

In [122]:
df['assessmentItemID'].str[4:7].value_counts().sort_values(ascending=False) # 중분류별 실제 유저들이 푼 개수

017    17739
009    17555
011    17029
003    16970
007    16826
018    16698
013    16517
015    16436
005    16325
057    16233
019    16050
025    15985
033    15806
047    15511
016    15448
023    15391
034    15081
049    14999
045    14952
055    14920
029    14920
001    14911
027    14851
032    14759
044    14758
021    14751
051    14674
128    14509
127    14473
043    14307
107    14301
031    14284
022    14241
067    14226
024    14127
026    14108
131    14068
109    14032
028    13966
103    13938
087    13936
061    13929
105    13898
014    13869
079    13838
059    13832
030    13792
058    13716
099    13710
071    13698
063    13672
073    13671
101    13662
091    13654
052    13634
020    13603
036    13594
065    13523
039    13469
069    13465
111    13442
135    13381
053    13258
124    13199
115    13180
048    13176
126    13131
123    13127
056    13119
035    13116
133    13115
083    13114
113    13080
077    13067
137    13008
117    12999
097    12943

In [98]:
def classify_2(x):
    x = int(x)
    if x<20: return 1
    elif x<40: return 2
    elif x<60: return 3
    elif x<80: return 4
    elif x<100: return 5
    elif x<120: return 6
    elif x<140: return 7
    elif x<160: return 8
    elif x<180: return 9
    elif x<200: return 10

In [99]:
df['2-2'] = df['2'].apply(lambda x: classify_2(x))

In [103]:
df.groupby('2-2')[['answerCode']].mean() # 왜 초반이 더 정답률이 낮은가...

,answerCode
2-2,
1,0.648943
2,0.631313
3,0.589522
4,0.642333
5,0.667013
6,0.657941
7,0.653556
8,0.697462
9,0.691842


## 소분류(7~10)

In [121]:
a['3'].value_counts() # 03을 제외하고 문제수가 줄어든다.

003    1532
001    1531
002    1525
004    1525
005    1482
006     859
007     574
008     298
009      80
010      22
011      14
012      11
013       1
Name: 3, dtype: int64

In [177]:
df.groupby('3')['answerCode'].count()

3
001    371755
002    370972
003    371640
004    369705
005    360607
006    197772
007    127121
008     67150
009     18417
010      5342
011      3352
012      2484
013       269
Name: answerCode, dtype: int64

In [120]:
df.groupby('3')['answerCode'].mean() # 뒷 문제일수록 난이도가 높다.

3
001    0.749916
002    0.720062
003    0.687773
004    0.663364
005    0.599134
006    0.555685
007    0.515399
008    0.457156
009    0.481729
010    0.527892
011    0.480609
012    0.370370
013    0.200743
Name: answerCode, dtype: float64

## 태그

In [164]:
b = df[['assessmentItemID','KnowledgeTag']].drop_duplicates()

In [170]:
b.groupby('KnowledgeTag').count().sort_values(by='assessmentItemID').describe()

,assessmentItemID
count,912.000000
mean,10.366228
std,6.622292
min,1.000000
25%,5.000000
50%,10.000000
75%,16.000000
max,41.000000


In [135]:
df.groupby('KnowledgeTag')['answerCode'].mean().sort_values()

KnowledgeTag
8911     0.188940
10332    0.250000
9125     0.257009
10590    0.268139
5763     0.278027
           ...   
1580     0.941896
1582     0.949280
1577     0.950461
7224     0.955022
1878     0.977778
Name: answerCode, Length: 912, dtype: float64

In [157]:
sum(df.groupby('KnowledgeTag')['answerCode'].mean().sort_values()[:100].index.tolist())//100

/tmp/ipykernel_11771/4229000699.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  sum(df.groupby('KnowledgeTag')['answerCode'].mean().sort_values()[:100].index.tolist())//100


6251

In [159]:
sum(df.groupby('KnowledgeTag')['answerCode'].mean().sort_values()[400:500].index.tolist())//100

/tmp/ipykernel_11771/2913438581.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  sum(df.groupby('KnowledgeTag')['answerCode'].mean().sort_values()[400:500].index.tolist())//100


4330

In [158]:
sum(df.groupby('KnowledgeTag')['answerCode'].mean().sort_values()[-100:].index.tolist())//100

/tmp/ipykernel_11771/1508012080.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  sum(df.groupby('KnowledgeTag')['answerCode'].mean().sort_values()[-100:].index.tolist())//100


5535

In [145]:
df.groupby('KnowledgeTag')['answerCode'].count().sort_values() # 태그별 문제 수

KnowledgeTag
2613       44
7125       45
2871       46
4978       46
5142       47
        ...  
8127     9006
7621     9709
2010    10176
7600    10734
7597    12892
Name: answerCode, Length: 912, dtype: int64

In [143]:
df.groupby('KnowledgeTag')['answerCode'].mean().sort_values().index

Int64Index([ 8911, 10332,  9125, 10590,  5763,  9167,  2725,  8897,  9498,
             4964,
            ...
             2129,  5844,  1465,  6806,  5846,  1580,  1582,  1577,  7224,
             1878],
           dtype='int64', name='KnowledgeTag', length=912)

In [140]:
df.groupby('KnowledgeTag')['answerCode'].mean().sort_values()

KnowledgeTag
8911     0.188940
10332    0.250000
9125     0.257009
10590    0.268139
5763     0.278027
           ...   
1580     0.941896
1582     0.949280
1577     0.950461
7224     0.955022
1878     0.977778
Name: answerCode, Length: 912, dtype: float64

In [260]:
df.groupby('assessmentItemID')['answerCode'].mean()

assessmentItemID
A010001001    0.930599
A010001002    0.958991
A010001003    0.905363
A010001004    0.949527
A010001005    0.886435
                ...   
A090074002    0.444444
A090074003    0.577778
A090074004    0.333333
A090074005    0.400000
A090074006    0.466667
Name: answerCode, Length: 9454, dtype: float64

### 결론 : 대분류, 소분류, 태그는 컬럼으로 충분히 넣어줄 법하다. 하지만 태그는 정리가 가능하다면 좋을듯

In [183]:
df[df[['userID','assessmentItemID']].duplicated()]

,userID,assessmentItemID,answerCode,Timestamp,KnowledgeTag,1,2,3,2-2
1397,1,A040155001,1,2020-08-17 09:56:27,2111,040,155,001,8
1398,1,A040155002,1,2020-08-17 09:57:06,2111,040,155,002,8
1399,1,A040155003,1,2020-08-17 09:58:03,2111,040,155,003,8
1400,1,A040155004,1,2020-08-17 09:58:29,2111,040,155,004,8
1401,1,A040155005,0,2020-08-17 09:59:40,2111,040,155,005,8
...,...,...,...,...,...,...,...,...,...
2257570,6977,A040167005,0,2020-10-26 13:58:52,2116,040,167,005,9
2264863,7329,A040198001,1,2020-10-12 12:08:49,2133,040,198,001,10
2264864,7329,A040198002,1,2020-10-12 12:08:53,2133,040,198,002,10
2264865,7329,A040198003,1,2020-10-12 12:09:02,2133,040,198,003,10


In [184]:
data_dir = '/opt/ml/input/data'
path1 = os.path.join(data_dir, "train_data.csv")
path2 = os.path.join(data_dir, "test_data.csv")
data1 = pd.read_csv(path1)
data2 = pd.read_csv(path2)

data = pd.concat([data1, data2])
data.drop_duplicates(
    subset=["userID", "assessmentItemID"], keep="last", inplace=True
)

In [186]:
train_data = data[data.answerCode >= 0]
test_data = data[data.answerCode < 0]

In [188]:
userid, itemid = (
    sorted(list(set(data.userID))),
    sorted(list(set(data.assessmentItemID))),
)
n_user, n_item = len(userid), len(itemid)

userid_2_index = {v: i for i, v in enumerate(userid)}
itemid_2_index = {v: i + n_user for i, v in enumerate(itemid)} # 왜 i + n_user?
id_2_index = dict(userid_2_index, **itemid_2_index) # 딕셔너리 합침

In [214]:
data[:2]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225


In [215]:
edge, label = [], []
for user, item, acode in zip(data.userID, data.assessmentItemID, data.answerCode):
    uid, iid = id_2_index[user], id_2_index[item]
    edge.append([uid, iid])
    label.append(acode)

edge = torch.LongTensor(edge).T
label = torch.LongTensor(label)

#dict(edge=edge.to(device), label=label.to(device))

NameError: name 'torch' is not defined

In [217]:
# edge

In [253]:
    data['main'] = data['assessmentItemID'].str[2]
    data['middle'] = data['assessmentItemID'].str[3:7]
    data['sub1'] = data['assessmentItemID'].str[7:10]

In [254]:
    mainid, middleid, subid = (
        sorted(list(set(data.main))),
        sorted(list(set(data.middle))),
        sorted(list(set(data.sub1)))
    )

In [255]:
n_mainid, n_middleid, n_subid = len(mainid), len(middleid), len(sub)

In [256]:
    main_2_index = {v: i for i, v in enumerate(mainid)}
    middle_2_index = {v: i + n_mainid for i, v in enumerate(middleid)}
    sub_2_index = {v: i + n_mainid + n_middleid for i, v in enumerate(subid)}

In [257]:
id_2_index = dict(main_2_index,**middle_2_index, **sub_2_index)

In [258]:
# id_2_index

{'1': 0,
 '2': 1,
 '3': 2,
 '4': 3,
 '5': 4,
 '6': 5,
 '7': 6,
 '8': 7,
 '9': 8,
 '0001': 9,
 '0002': 10,
 '0003': 11,
 '0004': 12,
 '0005': 13,
 '0006': 14,
 '0007': 15,
 '0008': 16,
 '0009': 17,
 '0010': 18,
 '0011': 19,
 '0012': 20,
 '0013': 21,
 '0014': 22,
 '0015': 23,
 '0016': 24,
 '0017': 25,
 '0018': 26,
 '0019': 27,
 '0020': 28,
 '0021': 29,
 '0022': 30,
 '0023': 31,
 '0024': 32,
 '0025': 33,
 '0026': 34,
 '0027': 35,
 '0028': 36,
 '0029': 37,
 '0030': 38,
 '0031': 39,
 '0032': 40,
 '0033': 41,
 '0034': 42,
 '0035': 43,
 '0036': 44,
 '0037': 45,
 '0038': 46,
 '0039': 47,
 '0040': 48,
 '0041': 49,
 '0042': 50,
 '0043': 51,
 '0044': 52,
 '0045': 53,
 '0046': 54,
 '0047': 55,
 '0048': 56,
 '0049': 57,
 '0050': 58,
 '0051': 59,
 '0052': 60,
 '0053': 61,
 '0054': 62,
 '0055': 63,
 '0056': 64,
 '0057': 65,
 '0058': 66,
 '0059': 67,
 '0060': 68,
 '0061': 69,
 '0062': 70,
 '0063': 71,
 '0064': 72,
 '0065': 73,
 '0066': 74,
 '0067': 75,
 '0068': 76,
 '0069': 77,
 '0070': 78,
 '0071': 7

In [227]:
data.sub1

0         001
1         002
2         003
3         004
4         005
         ... 
260109    001
260110    002
260111    003
260112    004
260113    005
Name: sub1, Length: 2476706, dtype: object

In [232]:
    userid, itemid = (
        sorted(list(set(data.userID))),
        sorted(list(set(data.assessmentItemID))),
    )
    
    mainid, middleid, subid = (
        sorted(list(set(data.main))),
        sorted(list(set(data.middle))),
        sorted(list(set(data.sub1)))
    )
    
    
    n_user, n_item = len(userid), len(itemid)
    n_mainid, n_middleid, n_subid = len(mainid), len(middleid), len(subid)

    userid_2_index = {v: i for i, v in enumerate(userid)}
    itemid_2_index = {v: i + n_user for i, v in enumerate(itemid)}
    
    main_2_index = {v: i + n_user + n_item for i, v in enumerate(mainid)}
    # middle_2_index = {v: i + n_user + n_item for i, v in enumerate(middleid)}
    sub_2_index = {v: i + n_user + n_item + n_mainid for i, v in enumerate(subid)}
    
    id_2_index = dict(userid_2_index, **itemid_2_index, **main_2_index, **sub_2_index)

In [234]:
len(id_2_index)

16918

In [237]:
import torch

In [244]:
    edge, label = [], []
    for user, item, acode, main, sub in zip(data.userID, data.assessmentItemID, data.answerCode, data.main, data.sub1):
        uid, iid = id_2_index[user], id_2_index[item]
        mainid = id_2_index[main]
        subid = id_2_index[sub]
        
        edge.append([uid, iid,mainid,subid])
        # edge.append([mainid,subid])
        label.append(acode)

In [238]:
    edge = torch.LongTensor(edge).T
    label = torch.LongTensor(label)

    # return dict(edge=edge.to(device), label=label.to(device))

In [245]:
edge

[[0, 12796, 16901, 16905],
 [0, 12797, 16901, 16906],
 [0, 12798, 16901, 16907],
 [0, 12799, 16901, 16908],
 [0, 12800, 16901, 16909],
 [0, 12801, 16901, 16911],
 [0, 12809, 16901, 16905],
 [0, 12810, 16901, 16906],
 [0, 12811, 16901, 16907],
 [0, 12812, 16901, 16908],
 [0, 12813, 16901, 16909],
 [0, 12814, 16901, 16910],
 [0, 12815, 16901, 16911],
 [0, 12823, 16901, 16905],
 [0, 12824, 16901, 16906],
 [0, 12825, 16901, 16907],
 [0, 12826, 16901, 16908],
 [0, 12827, 16901, 16909],
 [0, 12828, 16901, 16910],
 [0, 12829, 16901, 16911],
 [0, 12837, 16901, 16905],
 [0, 12838, 16901, 16906],
 [0, 12839, 16901, 16907],
 [0, 12840, 16901, 16908],
 [0, 12841, 16901, 16909],
 [0, 12842, 16901, 16910],
 [0, 12843, 16901, 16911],
 [0, 15337, 16903, 16905],
 [0, 15338, 16903, 16906],
 [0, 15339, 16903, 16907],
 [0, 15340, 16903, 16908],
 [0, 15341, 16903, 16909],
 [0, 15342, 16903, 16910],
 [0, 12851, 16901, 16905],
 [0, 12852, 16901, 16906],
 [0, 12853, 16901, 16907],
 [0, 12854, 16901, 16908],
 